<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/AA_GRUPO3/blob/main/LAB03_Pr4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importación de librerías a utilizar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from numpy.core.function_base import linspace
from math import floor
from math import ceil
from numpy.ma.core import size

from sklearn import tree
from sklearn.datasets import load_iris
import graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

#Descarga y lectura de datos

In [2]:
# Lectura de la base de datos
filename = 'lluvias.csv'
data = pd.read_csv(filename, sep = ',')

###Revisión y limpieza de datos
Observamos que tenemos un dataset con un total de 25551 registros y solo existen datos NaN en 3 de ellos, es decir, está bastante completo.

In [3]:
# Revisión de los datos
print(data.shape)

data.info()
data.head(10)

(25551, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25551 entries, 0 to 25550
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DATE    25551 non-null  object 
 1   PRCP    25548 non-null  float64
 2   TMAX    25551 non-null  int64  
 3   TMIN    25551 non-null  int64  
 4   RAIN    25548 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 998.2+ KB


,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,0.47,51,42,True
1,1948-01-02,0.59,45,36,True
2,1948-01-03,0.42,45,35,True
3,1948-01-04,0.31,45,34,True
4,1948-01-05,0.17,45,32,True
5,1948-01-06,0.44,48,39,True
6,1948-01-07,0.41,50,40,True
7,1948-01-08,0.04,48,35,True
8,1948-01-09,0.12,50,31,True
9,1948-01-10,0.74,43,34,True


Decidimos eliminar los Nan ya que son solo 3 y no será significativo para el estudio que vamos a realizar. Y quitamos los duplicados, que como podemos observar, en este caso no los hay.

In [4]:
# Elimino los Nan.
data = data.dropna()

# Quitamos los duplicados
data = data.drop_duplicates()

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25548 entries, 0 to 25550
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DATE    25548 non-null  object 
 1   PRCP    25548 non-null  float64
 2   TMAX    25548 non-null  int64  
 3   TMIN    25548 non-null  int64  
 4   RAIN    25548 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ MB


Nos damos cuenta de que el valor máximo de la temperatura es muy alto, por lo que entendemos que los registros están en grados Fahrenheit. 

Realizamos la conversión de todas las temperaturas a grados Celsius, con la fórmula de conversión.

In [5]:
temp_max = data["TMAX"].max()

print('La temperatura máxima encontrada es', temp_max,'entendemos que estamos en Fº y queremos trabajar en Cº.')

La temperatura máxima encontrada es 103 entendemos que estamos en Fº y queremos trabajar en Cº.


In [6]:
data["TMIN"] = (data.TMIN - 32)*(5/9)

data["TMAX"] = (data.TMAX - 32)*(5/9)

temp_max = data["TMAX"].max().round(2)

temp_min = data["TMIN"].min().round(2)

print('La temperatura máxima en grados Celsius es', temp_max,'Cº.')

La temperatura máxima en grados Celsius es 39.44 Cº.


Imprimimos los primeros 40 registros para comprobar que se hayan modificado correctamente.

In [7]:
data.head(40)

,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,0.47,10.555556,5.555556,True
1,1948-01-02,0.59,7.222222,2.222222,True
2,1948-01-03,0.42,7.222222,1.666667,True
3,1948-01-04,0.31,7.222222,1.111111,True
4,1948-01-05,0.17,7.222222,0.000000,True
5,1948-01-06,0.44,8.888889,3.888889,True
6,1948-01-07,0.41,10.000000,4.444444,True
7,1948-01-08,0.04,8.888889,1.666667,True
8,1948-01-09,0.12,10.000000,-0.555556,True
9,1948-01-10,0.74,6.111111,1.111111,True


Categorizamos las columnas 'TMIN', 'TMAX' y 'PRCP' tal y como se pide en el enunciado del ejercicio.

'TMIN' tomará las etiquetas 'Baja', 'Media' y 'Alta', en función de los rangos de temperatura < 10, 10 >= temeperatura <= 20 y temperatura > 20.

In [8]:
conditions = [
          (data['TMIN'] < 10),
          ((data['TMIN'] >= 10) & (data['TMIN'] <= 20)),
          (data['TMIN'] > 20)
             ]

#define results
results = ['Baja', 'Media', 'Alta']

#create new column based on conditions in column1 and column2
data['TMIN'] = np.select(conditions, results)

'TMAX' se categoriza igual que 'TMIN'.

In [9]:
conditions = [
          (data['TMAX'] < 10),
          ((data['TMAX'] >= 10) & (data['TMAX'] <= 20)),
          (data['TMAX'] > 20)
             ]

#define results
results = ['Baja', 'Media', 'Alta']

#create new column based on conditions in column1 and column2
data['TMAX'] = np.select(conditions, results)

In [10]:
# Vemos cuantos registros de cada etiqueta tiene TMIN

data.value_counts(data['TMIN'])

TMIN
Baja     17000
Media     8543
Alta         5
dtype: int64

In [11]:
conditions = [
          (data['PRCP'] < 1),
          ((data['PRCP'] >= 1) & (data['PRCP'] <= 2)),
          (data['PRCP'] > 2)
             ]

#define results
results = ['Poco', 'Normal', 'Mucho']

#create new column based on conditions in column1 and column2
data['PRCP'] = np.select(conditions, results)

Vemos los primeros 20 registros para comprobar que las tres columnas han sido correctamente parametrizadas.

In [12]:
data.head(20)

,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,Poco,Media,Baja,True
1,1948-01-02,Poco,Baja,Baja,True
2,1948-01-03,Poco,Baja,Baja,True
3,1948-01-04,Poco,Baja,Baja,True
4,1948-01-05,Poco,Baja,Baja,True
5,1948-01-06,Poco,Baja,Baja,True
6,1948-01-07,Poco,Media,Baja,True
7,1948-01-08,Poco,Baja,Baja,True
8,1948-01-09,Poco,Media,Baja,True
9,1948-01-10,Poco,Baja,Baja,True


In [13]:
data.value_counts(data['PRCP'])

PRCP
Poco      25189
Normal      327
Mucho        32
dtype: int64

#Clasificación basada en redes Bayesianas

Contamos cuantos registros tienen como clasificación True y cuantos False y lo guardamos en variables, aunque luego solo utilizaremos las de True.

Creamos un dataset de entrenamiento que contiene el 70% de los registros del dataset original y otro de test que contiene el 30% restante para poder calcular el error al final.

In [169]:
data_train = data[0:round(0.7*len(data))]

data_test = data[round(0.7*len(data)):]

# Guardamos la frecuencia del valor 'True' y 'False' en variables
total_rain_true  = data_train['RAIN'].value_counts()[True]
total_rain_false = data_train['RAIN'].value_counts()[False]

# Total de registros
total_registros = total_rain_true + total_rain_false

# P(True)
prob_true = total_rain_true/total_registros
# data_train.value_counts(data_train['RAIN']

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17884 entries, 0 to 17883
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   DATE    17884 non-null  object
 1   PRCP    17884 non-null  object
 2   TMAX    17884 non-null  object
 3   TMIN    17884 non-null  object
 4   RAIN    17884 non-null  object
dtypes: object(5)
memory usage: 838.3+ KB


Creamos un nuevo dataset que contiene solo los registros clasificados como 'True', para poder sacar el resto de frecuencias. 

In [172]:
data_rain_true = data_train.query('RAIN != @False')

Frecuencias de la columna de precipitaciones de los registros clasificados como 'True' en el dataset.

Son clasificados según las diferentes categorías que puede tomar, es decir, 'Poco', 'Normal' y 'Mucho'. 

In [173]:
# Contamos la frecuencia de los valores en la columna de clase 'PRCP' con 'RAIN' == 'True'

total_p_PRCP = data_rain_true['PRCP'].value_counts()['Poco']
total_n_PRCP = data_rain_true['PRCP'].value_counts()['Normal']
total_m_PRCP = data_rain_true['PRCP'].value_counts()['Mucho']

print(total_m_PRCP)

19


Frecuencias de la columna de temperatura máxima de los registros clasificados como 'True' en el dataset.

Son clasificados según las diferentes categorías que puede tomar, es decir, 'Baja', 'Media' y 'Alta'.


In [174]:
# Guardamos la frecuencia de los valores 'Baja', 'Media' y 'Alta' en variables con 'RAIN' == 'True'
total_b_TMAX = data_rain_true['TMAX'].value_counts()['Baja']
total_m_TMAX = data_rain_true['TMAX'].value_counts()['Media']
total_a_TMAX = data_rain_true['TMAX'].value_counts()['Alta']

print(total_a_TMAX)

407


Vemos que no hay ningún registro de 'TMIN' alta en el dataset que contiene los registros clasificados como 'True', por lo que lo pondremos directamente como 0.

In [175]:
data_rain_true.value_counts(data_rain_true['TMIN'])

TMIN
Baja     6052
Media    1509
dtype: int64

Frecuencias de la columna de temperatura mínima de los registros clasificados como 'True' en el dataset.

Son clasificados según las diferentes categorías que puede tomar, es decir, 'Baja', 'Media' y 'Alta'.

In [176]:
# Contamos la frecuencia de los valores en la columna de clase 'TMAX' con 'RAIN' == 'True'
total_b_TMIN = data_rain_true['TMIN'].value_counts()['Baja']
total_m_TMIN = data_rain_true['TMIN'].value_counts()['Media']
# No hay ninguna 'Alta'
total_a_TMIN = 0

print(total_b_TMIN)

6052


###Probamos como clasifica según las probabilidades generadas

Creamos un dataset nuevo con los registros de los 3 días posteriores a los que ya tenemos en el dataset inicial, con sus correspondientes datos de Precipitaciones, Temperatura máxima y Temperatura mínima.

Se corresponden con los días '15-12-2017', '16-12-2017' y '17-12-2017'.

In [177]:
# Creamos un dataframe con los nuevos registros
registros_nuevos = pd.DataFrame({'DATE': ['15-12-2017', '16-12-2017', '17-12-2017'], 'PRCP': ['Poco', 'Mucho', 'Normal'], 'TMAX': ['Media', 'Baja', 'Alta'], 'TMIN': ['Baja', 'Media', 'Baja'], 'RAIN': [False, True, False]})

registros_nuevos.dtypes

DATE    object
PRCP    object
TMAX    object
TMIN    object
RAIN      bool
dtype: object

Utilizamos un bucle for que recorra el dataset nuevo. Se devuelve por cada registro, la probabilidad de que 'RAIN' == 'True', es decir, la probabilidad de que llueva.

In [178]:
# Buscamos los valores de los atributos para poder realizar la clasificación basada en redes bayesianas

# Iteramos sobre las filas del dataframe nuevo con un bucle for
def clasificacion_bayesiana(data_set):
  total = 0
  correctos = 0
  for indice, fila in data_set.iterrows():
      print('\nProbabilidades del día', fila['DATE'],'\n')
      
      # Probabilidades de 'RAIN == True' en función de las PRCP.
      if data_set.loc[data_set.index == indice, 'PRCP'].apply(lambda x: 'Poco' in x).item():
        probabilidad_PRCP = total_p_PRCP/total_rain_true
      elif data_set.loc[data_set.index == indice, 'PRCP'].apply(lambda x: 'Normal' in x).item():
        probabilidad_PRCP = total_n_PRCP/total_rain_true
      else:
        probabilidad_PRCP = total_m_PRCP/total_rain_true
      print(probabilidad_PRCP)

      # Probabilidades de 'RAIN == True' en función de la TMAX.
      if data_set.loc[data_set.index == indice, 'TMAX'].apply(lambda x: 'Baja' in x).item():
        probabilidad_TMAX = total_b_TMAX/total_rain_true
      elif data_set.loc[data_set.index == indice, 'TMAX'].apply(lambda x: 'Media' in x).item():
        probabilidad_TMAX = total_m_TMAX/total_rain_true
      else:
        probabilidad_TMAX = total_a_TMAX/total_rain_true
      print(probabilidad_TMAX)

      # Probabilidades de 'RAIN == True' en función de la TMIN.
      if data_set.loc[data_set.index == indice, 'TMIN'].apply(lambda x: 'Baja' in x).item():
        probabilidad_TMIN = total_b_TMIN/total_rain_true
      elif data_set.loc[data_set.index == indice, 'TMIN'].apply(lambda x: 'Media' in x).item():
        probabilidad_TMIN = total_m_TMIN/total_rain_true
      else:
        probabilidad_TMIN = total_a_TMIN/total_rain_true
      print(probabilidad_TMIN)

      probabilidad_lluvia = probabilidad_PRCP*probabilidad_TMAX*probabilidad_TMIN*prob_true
      print('\nLa probabilidad de lluvia del día', fila['DATE'],' es de', probabilidad_lluvia,'\n')
      total = total + 1

      if fila['RAIN']:
        if probabilidad_lluvia > 0.5:
          correctos = correctos + 1
      else:
        if probabilidad_lluvia <= 0.5:
          correctos = correctos + 1
  error = (total - correctos)/total
  print('El error es de', error)

In [183]:
clasificacion_bayesiana(registros_nuevos)


Probabilidades del día 15-12-2017 

0.9690517127364106
0.6120883481021029
0.8004232244412115

La probabilidad de lluvia del día 15-12-2017  es de 0.2007221609301009 


Probabilidades del día 16-12-2017 

0.0025128951196931623
0.334082793281312
0.19957677555878853

La probabilidad de lluvia del día 16-12-2017  es de 7.08358402148509e-05 


Probabilidades del día 17-12-2017 

0.02843539214389631
0.05382885861658511
0.8004232244412115

La probabilidad de lluvia del día 17-12-2017  es de 0.0005179748235911197 

El error es de 0.3333333333333333


###Cálculo de error

Creamos un dataset de entrenamiento que contiene el 70% de los registros del dataset original y otro de test que contiene el 30% restante para poder calcular el error.

In [114]:
registros_nuevos.dtypes

DATE    object
PRCP    object
TMAX    object
TMIN    object
RAIN    object
dtype: object

In [180]:
clasificacion_bayesiana(data_test)

Streaming output truncated to the last 5000 lines.
0.19957677555878853

La probabilidad de lluvia del día 2016-06-07  es de 0.004401358084561914 


Probabilidades del día 2016-06-08 

0.9690517127364106
0.6120883481021029
0.19957677555878853

La probabilidad de lluvia del día 2016-06-08  es de 0.050047875221996406 


Probabilidades del día 2016-06-09 

0.9690517127364106
0.6120883481021029
0.19957677555878853

La probabilidad de lluvia del día 2016-06-09  es de 0.050047875221996406 


Probabilidades del día 2016-06-10 

0.9690517127364106
0.6120883481021029
0.8004232244412115

La probabilidad de lluvia del día 2016-06-10  es de 0.2007221609301009 


Probabilidades del día 2016-06-11 

0.9690517127364106
0.6120883481021029
0.19957677555878853

La probabilidad de lluvia del día 2016-06-11  es de 0.050047875221996406 


Probabilidades del día 2016-06-12 

0.9690517127364106
0.05382885861658511
0.19957677555878853

La probabilidad de lluvia del día 2016-06-12  es de 0.004401358084561914 



###Conclusiones

Lo comparamos con las etiquetas verdaderas para sacar conclusiones y el error de nuestro modelo.

El error con aquellos registros que tienen como Precipitaciones la etiqueta 'Mucho', es muy grande. Esto es debido a que solo hay 32 registros con dicha etiqueta. Aunque como vemos a continuación en el conteo, todos pertenecen a registros donde SÍ ha llovido, la probabilidad de dicha precipitación es de 0.334 (19/7561), es decir, un número muy pequeño, por lo que la probabilidad de que llueva que devuelve el modelo será también muy pequeña.

In [22]:
data_rain_true.value_counts(data_rain_true['PRCP'])

PRCP
Poco      10541
Normal      327
Mucho        32
dtype: int64

In [23]:
data.value_counts(data['PRCP'])

PRCP
Poco      25189
Normal      327
Mucho        32
dtype: int64